Part 1

In [1]:
import re
num_pattern = r'\d+'
symbol_pattern = r'[^\d|\.]{1}'

def pt1(filename):
    with open(filename, 'r') as f:
        data = f.read().splitlines()

    numlists = [re.finditer(num_pattern, line) for line in data]
    symbolrngs = [set(
            range((s := symbol.start()) - 1, s + 2)
            for symbol in re.finditer(symbol_pattern, line)
        )
        for line in data
    ]

    sumnums = []
    for i, (nums, rngs) in enumerate(zip(numlists, symbolrngs)):
        allrngs = set.union(
            rngs,
            symbolrngs[i-1] if i - 1 >= 0 else set(),
            symbolrngs[i+1] if i + 1 < len(data) else set()
        )
        for num in nums:
            add = False
            for rng in allrngs:
                lower, upper = num.span()
                add |= lower in rng
                add |= upper - 1 in rng
            if add:
                sumnums.append(int(num.group()))
        
    return sum(sumnums)

pt1('test.txt'), pt1('input.txt')

(4361, 532331)

Part 2

In [2]:
import re
gear_pattern = r'\*'


def check_horizontal(data, i, j):
    h = []
    for direction in (-1, 1):
        y = j + direction
        while y in range(len(data[i])) and data[i][y].isdigit():
            y += direction
        if y != j + direction:
            h.append(
                int(data[i][y+1:j]) if direction < 0 else int(data[i][j+1:y])
            )
    return h


def get_neighbors(data, i, j):
    neighbors = check_horizontal(data, i, j)

    # check above/below
    for vert in (-1, 1):
        x = i + vert
        # check for one centered number
        if x in range(len(data)) and data[x][j].isdigit():
            y1 = j - 1
            while data[x][y1].isdigit():
                y1 -= 1
            y1 += 1

            y2 = j + 1
            while data[x][y2].isdigit():
                y2 += 1

            neighbors.append(int(data[x][y1:y2]))
            
        #check for multiple off-center
        elif x in range(len(data)):
            neighbors += check_horizontal(data, i + vert, j)

    return neighbors


def pt2(filename):
    with open(filename, 'r') as f:
        data = f.read().splitlines()

    gearlists = [
        [gear.start() for gear in re.finditer(gear_pattern, line)]
        for line in data
    ]

    sumnums = []
    for i, gears in enumerate(gearlists):
        for gear in gears:
            neighbors = get_neighbors(data, i, gear)
            if len(neighbors) == 2:
                sumnums.append(neighbors[0] * neighbors[1])
        
    return sum(sumnums)

pt2('test.txt'), pt2('input.txt')

(467835, 82301120)